In [ ]:
https://www.tensorflow.org/tutorials/keras/keras_tuner

## Introduction to the Keras Tuner

### Overview

The Keras Tuner is a library that helps you pick the optimal set of hyperparameters for your TensorFlow program. The process of selecting the right set of hyperparameters for your machine learning (ML) application is called hyperparameter tuning or hypertuning.

Hyperparameters are the variables that govern the training process and the topology of an ML model. These variables remain constant over the training process and directly impact the performance of your ML program. Hyperparameters are of two types:

<ol>
    <li><b>Model hyperparameters</b> which influence model selection such as the number and width of hidden layers</li>
    <li><b>Algorithm hyperparameters</b> which influence the speed and quality of the learning algorithm such as the learning rate for Stochastic Gradient Descent (SGD) and the number of nearest neighbors for a k Nearest Neighbors (KNN) classifier</li>
</ol>

In this tutorial, you will use the Keras Tuner to perform hypertuning for an image classification application.

### Setup

In [1]:
import tensorflow as tf
from tensorflow import keras

2024-05-31 03:36:35.756151: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Install and import the Keras Tuner.

In [2]:
!pip install -q -U keras-tuner


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
import keras_tuner as kt

### Download and prepare the dataset

In this tutorial, you will use the Keras Tuner to find the best hyperparameters for a machine learning model that classifies images of clothing from the <a href="https://github.com/zalandoresearch/fashion-mnist">Fashion MNIST dataset</a>.

Load the data.

In [4]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [5]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

### Define the model

When you build a model for hypertuning, you also define the hyperparameter search space in addition to the model architecture. The model you set up for hypertuning is called a hypermodel.

You can define a hypermodel through two approaches:

<ul>
    <li>By using a model builder function</li>
    <li>By subclassing the HyperModel class of the Keras Tuner API</li>
</ul>

You can also use two pre-defined <a href="https://keras.io/api/keras_tuner/hypermodels/">HyperModel</a> classes - <a href="https://keras.io/api/keras_tuner/hypermodels/hyper_xception/">HyperXception</a> and <a href="https://keras.io/api/keras_tuner/hypermodels/hyper_resnet/">HyperResNet</a> for computer vision applications.

In this tutorial, you use a model builder function to define the image classification model. The model builder function returns a compiled model and uses hyperparameters you define inline to hypertune the model.

In [6]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

### Instantiate the tuner and perform hypertuning

Instantiate the tuner to perform the hypertuning. The Keras Tuner has four tuners available - RandomSearch, Hyperband, BayesianOptimization, and Sklearn. In this tutorial, you use the <a href="https://arxiv.org/pdf/1603.06560.pdf">Hyperband</a> tuner.

To instantiate the Hyperband tuner, you must specify the hypermodel, the objective to optimize and the maximum number of epochs to train (max_epochs).

In [7]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

The Hyperband tuning algorithm uses adaptive resource allocation and early-stopping to quickly converge on a high-performing model. This is done using a sports championship style bracket. The algorithm trains a large number of models for a few epochs and carries forward only the top-performing half of models to the next round. Hyperband determines the number of models to train in a bracket by computing 1 + logfactor(max_epochs) and rounding it up to the nearest integer.

Create a callback to stop training early after reaching a certain value for the validation loss.

In [8]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

Run the hyperparameter search. The arguments for the search method are the same as those used for tf.keras.model.fit in addition to the callback above.

In [9]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 00m 30s]
val_accuracy: 0.8681666851043701

Best val_accuracy So Far: 0.893833339214325
Total elapsed time: 00h 08m 22s

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 320 and the optimal learning rate for the optimizer
is 0.001.



### Train the model
Find the optimal number of epochs to train the model with the hyperparameters obtained from the search.

In [10]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.4922 - accuracy: 0.8234 - val_loss: 0.4171 - val_accuracy: 0.8465
Epoch 2/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3705 - accuracy: 0.8649 - val_loss: 0.3568 - val_accuracy: 0.8720
Epoch 3/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3343 - accuracy: 0.8761 - val_loss: 0.3464 - val_accuracy: 0.8686
Epoch 4/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3086 - accuracy: 0.8854 - val_loss: 0.3281 - val_accuracy: 0.8811
Epoch 5/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2901 - accuracy: 0.8929 - val_loss: 0.3429 - val_accuracy: 0.8757
Epoch 6/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2718 - accuracy: 0.8994 - val_loss: 0.3092 - val_accuracy: 0.8892
Epoch 7/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2571 - accuracy: 0.9043 - val_loss: 0.3350 - val_accuracy:

Re-instantiate the hypermodel and train it with the optimal number of epochs from above.

In [11]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/29
1500/1500 [==============================] - 5s 3ms/step - loss: 0.4960 - accuracy: 0.8241 - val_loss: 0.4057 - val_accuracy: 0.8543
Epoch 2/29
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3700 - accuracy: 0.8650 - val_loss: 0.3593 - val_accuracy: 0.8724
Epoch 3/29
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3305 - accuracy: 0.8794 - val_loss: 0.3474 - val_accuracy: 0.8760
Epoch 4/29
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3050 - accuracy: 0.8879 - val_loss: 0.3571 - val_accuracy: 0.8734
Epoch 5/29
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2873 - accuracy: 0.8940 - val_loss: 0.3242 - val_accuracy: 0.8853
Epoch 6/29
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2713 - accuracy: 0.8987 - val_loss: 0.3479 - val_accuracy: 0.8776
Epoch 7/29
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2583 - accuracy: 0.9062 - val_loss: 0.3354 - val_accuracy:

To finish this tutorial, evaluate the hypermodel on the test data.

In [12]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 0s 1ms/step - loss: 0.4524 - accuracy: 0.8868
[test loss, test accuracy]: [0.4523850083351135, 0.8867999911308289]


The my_dir/intro_to_kt directory contains detailed logs and checkpoints for every trial (model configuration) run during the hyperparameter search. If you re-run the hyperparameter search, the Keras Tuner uses the existing state from these logs to resume the search. To disable this behavior, pass an additional overwrite=True argument while instantiating the tuner.

### Summary

In this tutorial, you learned how to use the Keras Tuner to tune hyperparameters for a model. To learn more about the Keras Tuner, check out these additional resources:

<ul>
    <li><a href="https://blog.tensorflow.org/2020/01/hyperparameter-tuning-with-keras-tuner.html?_gl=1*10oe7kj*_ga*MTYzMjQ3Mzk5LjE3MTYxNjIyNTA.*_ga_W0YLR4190T*MTcxNzEyNjQxMS4xNi4xLjE3MTcxMjcyNTguMC4wLjA.">Keras Tuner on the TensorFlow blog</a></li>
    <li><a href="https://keras-team.github.io/keras-tuner/">Keras Tuner website</a></li>
</ul>

Also check out the <a href="https://www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams">HParams Dashboard</a> in TensorBoard to interactively tune your model hyperparameters.